## 準備資料集

In [1]:
import numpy as np

x_data = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=np.float32)

y_data = np.array([0, 0, 0, 1], dtype=np.float32)

print("x_data shape:", x_data.shape)
print("y_data shape:", y_data.shape)

x_data shape: (4, 2)
y_data shape: (4,)


## 建立模型

In [2]:
class Perceptron:
    def __init__(self, input_size, learning_rate=0.1):
        # 隨機初始化w與b
        self.w = np.random.randn(input_size)
        self.b = np.random.randn()
        # 設定學習率
        self.lr = learning_rate        

    def forward(self, x, y):
        z = np.dot(x, self.w) + self.b         # 公式 2.1：z = XW + b
        y_hat = 1 / (1 + np.exp(-z))           # 公式 2.2：y ̂ = f(z)
        loss = self.loss(y_hat, y)             # 公式 2.6：MSE Loss

        return loss, y_hat
       

    def loss(self, y_hat, y): # MSE loss 實作
        return 0.5 * np.mean((y_hat - y) ** 2) # 公式 2.6

    def backward(self, x, y, y_hat):
        dL_dyhat = y_hat - y                                   # ∂L/∂ŷ = ŷ - y（公式2.7）
        dyhat_dz = y_hat * (1 - y_hat)                         # ∂ŷ/∂z = ŷ ⊙ (1 - ŷ)
        delta = dL_dyhat * dyhat_dz                            # ∂L/∂z = (ŷ - y) ⊙ ŷ ⊙ (1 - ŷ)

        grad_w = np.dot(x.T, delta) / len(x)                   # ∂L/∂W = 1/n X^T [(ŷ - y) ⊙ ŷ ⊙ (1 - ŷ)]（公式2.9）
        grad_b = np.sum(delta) / len(x)                        # ∂L/∂b = 1/n 1^T [(ŷ - y) ⊙ ŷ ⊙ (1 - ŷ)]（公式2.10）

        # 優化器更新權重 (公式2.11)
        self.w -= self.lr * grad_w
        self.b -= self.lr * grad_b

    def predict(self, x):
        z = np.dot(x, self.w) + self.b
        y_hat = 1 / (1 + np.exp(-z))

        return y_hat, (y_hat >= 0.5).astype(int)
    
model = Perceptron(input_size=x_data.shape[1], learning_rate=0.1)

## 訓練模型

In [3]:
def train(model, x_data, y_data, epochs=10000):
    for epoch in range(epochs):
        loss, y_hat = model.forward(x_data, y_data)  # 前向傳播 + 損失計算
        model.backward(x_data, y_data, y_hat)        # 反向傳播 + 參數更新

        if (epoch + 1) % 1000 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss:.6f}")
            
train(model, x_data, y_data, epochs=10000)

Epoch 1000, Loss: 0.046753
Epoch 2000, Loss: 0.029379
Epoch 3000, Loss: 0.021057
Epoch 4000, Loss: 0.016153
Epoch 5000, Loss: 0.012961
Epoch 6000, Loss: 0.010743
Epoch 7000, Loss: 0.009127
Epoch 8000, Loss: 0.007905
Epoch 9000, Loss: 0.006952
Epoch 10000, Loss: 0.006191


## 推理結果

In [4]:
print("\nPrediction Results:")
results = model.predict(x_data)
for i in range(len(x_data)):
    print(f"Input: {x_data[i]}, Output: {results[0][i]:.4f}, Predicted Class: {results[1][i]}")


Prediction Results:
Input: [0. 0.], Output: 0.0031, Predicted Class: 0
Input: [0. 1.], Output: 0.1199, Predicted Class: 0
Input: [1. 0.], Output: 0.1199, Predicted Class: 0
Input: [1. 1.], Output: 0.8560, Predicted Class: 1
